the mothod of label converting is optimised

In [194]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import numpy as np
import torch

# convert sequence to matrix
def one_hot(sequence):
    nucl_list = ['A', 'T', 'C', 'G']
    nucl_dic = dict((int, nucl) for nucl, int in enumerate(nucl_list))
    _seq_data = [nucl_dic[base] for base in sequence]
    result = np.zeros((len(sequence), 4))
    for _, base in enumerate(_seq_data):
        result[_, base] = 1
    return(result)

# convert label to tensor
def label2tensor(labels):
    labels_dict = dict((key, value) for value, key in enumerate(labels))
    labels = [labels_dict[i] for i in labels]
    result = np.zeros((len(labels), 4))
    
    for _, base in enumerate(labels):
        result[_, base] = 1
    # convert result to tensor
    result = torch.tensor(result).to(torch.float32)
    
    return result

# cut off sequence to 2k segment
def seq2onehot(sequence_file):
    
    seq_label = [] # the list is including seq corresponding accession
    seq_data = [] # a sequence is converted to a list of 2k sub-seq
    
    for seq_record in SeqIO.parse(sequence_file, 'fasta'):
        # virus 2k DNA sequence segments
        new_record = []
        # get sequence data
        seq = seq_record.seq
        # remove N
        seq = str(seq).replace('N', '')
        
        for i in range(0, len(seq), 500):
            if (i + 2000) > len(seq):
                new_seq = seq[-2000:]
                record = SeqRecord(new_seq, id=seq_record.id)
                new_record.append(record)
                break
            new_seq = seq[i:i+2000]
            record = SeqRecord(new_seq, id=seq_record.id)
            new_record.append(record)
 
        seq_data.append(torch.tensor(np.array([one_hot(segment) for segment in new_record], dtype=np.float32)))

        seq_label.append(seq_record.id)
    
    seq_label = label2tensor(seq_label)
        
    return seq_label, seq_data

In [95]:
data_path = '/home/ouconstand/data/Virus_Host/seq_data/seq_train.fasta'

In [195]:
labels, train_data = seq2onehot(data_path)

In [196]:
for i in range(4):
    print(labels[i].shape)

torch.Size([4])
torch.Size([4])
torch.Size([4])
torch.Size([4])


In [197]:
for i in range(4):
    print(train_data[i].shape)

torch.Size([56, 2000, 4])
torch.Size([56, 2000, 4])
torch.Size([58, 2000, 4])
torch.Size([52, 2000, 4])


In [209]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        # three convolutional layers and a maxpool layer
        self.convs_pool_layers = nn.Sequential(
            nn.Conv1d(in_channels=4, out_channels=32, kernel_size=3),
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5),
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=7),
            nn.MaxPool1d(kernel_size=2)
        )
        # it is not necessary to build a softmax layer because of the loss function including a softmax layer
        self.fc = nn.Linear(128, 256)
        self.relu = nn.ReLU()
        
        
    def forward(self, x):
        x = x.permute(0,2,1)
        x = [self.convs_pool_layers(i) for i in x]
        x = torch.cat(x, 1)
        x = self.relu(x)
        
        return x

In [210]:
# input (batch_size, seq_length, feature_num)
model = MyModel()
print(model)

MyModel(
  (convs_pool_layers): Sequential(
    (0): Conv1d(4, 32, kernel_size=(3,), stride=(1,))
    (1): Conv1d(32, 64, kernel_size=(5,), stride=(1,))
    (2): Conv1d(64, 128, kernel_size=(7,), stride=(1,))
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=128, out_features=256, bias=True)
  (relu): ReLU()
)


In [211]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [214]:

# zero the parameter gradient
optimizer.zero_grad() # ?

# forward + backward + optimize
output = model(train_data[0])
loss = criterion(output, labels)
loss.backward()
optimizer.step()


torch.Size([128, 55664])

In [215]:
output

tensor([[0.0000, 0.1779, 0.3427,  ..., 0.1087, 0.0263, 0.1584],
        [0.0729, 0.0000, 0.1222,  ..., 0.0890, 0.0000, 0.0135],
        [0.0471, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.2689, 0.1754, 0.0867,  ..., 0.1421, 0.0692, 0.0349],
        [0.1692, 0.2337, 0.1117,  ..., 0.0000, 0.0983, 0.1940],
        [0.1444, 0.0000, 0.0351,  ..., 0.0000, 0.0488, 0.0000]],
       grad_fn=<ReluBackward0>)